In [12]:
// Import necessary dependencies
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")


In [43]:
import com.fasterxml.jackson.annotation.JsonFormat
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.io.File
import java.time.Duration
import java.time.LocalTime


In [17]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}


In [25]:
import com.fasterxml.jackson.annotation.JsonProperty

data class Client(
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" or "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Convert "Yes"/"No" to Boolean
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    override fun toString(): String = clientId
}


In [19]:
data class Equipment(
    val name: String,
    val capacityKg: Double
) {
    override fun toString(): String = name
}


In [20]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId

@PlanningEntity
data class Truck(
    @PlanningId
    val name: String,
    val type: String, // "Multilift 4 essieux", "Multilift 4 essieux Tridem", "Camion-grue 4 essieux"
    val homeLocation: Location,
    val capacityKg: Double,
    val canAccessDifficultAreas: Boolean,
    val canEquipRemorque: Boolean,
    val possibleEquipments: List<Equipment>,
    val departureTime: String = "08:00"
) {
    @PlanningListVariable(valueRangeProviderRefs = ["clientRange"])
    var assignedClients: MutableList<Client> = mutableListOf()

    @PlanningVariable(valueRangeProviderRefs = ["equipmentRange"])
    var equipment: Equipment? = null

    // No-arg constructor required for Timefold
    constructor() : this("", "", Location(0.0, 0.0), 0.0, false, false, emptyList())

    override fun toString(): String = name
}


In [21]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            capacityConstraint(constraintFactory),
            equipmentCompatibilityConstraint(constraintFactory),
            accessDifficultyConstraint(constraintFactory),
            timeWindowConstraint(constraintFactory),
            minimizeDistanceConstraint(constraintFactory)
        )
    }

    // Hard constraint: Do not exceed truck capacity
    private fun capacityConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Truck::class.java)
            .filter { truck ->
                val totalLoad = truck.assignedClients.sumOf { it.volumeKg }
                totalLoad > truck.capacityKg
            }
            .penalizeLong(HardSoftLongScore.ONE_HARD) { truck ->
                val totalLoad = truck.assignedClients.sumOf { it.volumeKg }
                (totalLoad - truck.capacityKg).toLong()
            }
            .asConstraint("Truck capacity exceeded")
    }

    // Hard constraint: Equipment compatibility with client needs
    private fun equipmentCompatibilityConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Client::class.java)
            .filter { client ->
                val truck = getTruckAssignedToClient(client)
                if (truck == null || truck.equipment == null) true
                else !isEquipmentCompatible(client, truck.equipment!!)
            }
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("Equipment not compatible with client need")
    }

    // Hard constraint: Truck access difficulty
    private fun accessDifficultyConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Client::class.java)
            .filter { client ->
                client.accessDifficulty && !getTruckAssignedToClient(client)?.canAccessDifficultAreas!!
            }
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("Truck cannot access client's location")
    }

    // Soft constraint: Minimize total distance traveled
    private fun minimizeDistanceConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEach(Truck::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT) { truck ->
                var distance: Double = 0.0
                var previousLocation: Location = truck.homeLocation
                for (client in truck.assignedClients) {
                    distance += previousLocation.calcEuclideanDistanceTo(client.location)
                    previousLocation = client.location
                }
                distance += previousLocation.calcEuclideanDistanceTo(truck.homeLocation)
                (distance * 1000).toLong() // Multiply to convert to meters
            }
            .asConstraint("Minimize distance traveled")
    }

    // Hard constraint: Respect client time windows
    private fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        // For simplicity, this is left as a placeholder
        return constraintFactory.forEach(Client::class.java)
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("Client time window not respected")
    }

    // Helper methods
    private fun getTruckAssignedToClient(client: Client): Truck? {
        // Implementation to find the truck assigned to the client
        // For Timefold Solver, this would be done via shadow variables or reverse mapping
        // Placeholder implementation
        return null
    }

    private fun isEquipmentCompatible(client: Client, equipment: Equipment): Boolean {
        // Define logic to check if the equipment is compatible with client need
        when (client.need) {
            "Waste" -> return equipment.name in listOf("Par bennes 10 m3", "Par bennes 20 m3", "Par pont 13 m3")
            "Enrobé" -> return equipment.name == "Par thermos 15 à 18 to"
            "Grue" -> return equipment.name == "Crane"
            else -> return false
        }
    }
}


In [22]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore

@PlanningSolution
data class Schedule(
    @PlanningEntityCollectionProperty
    val trucks: List<Truck>,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "clientRange")
    val clients: List<Client>,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "equipmentRange")
    val equipments: List<Equipment>
) {
    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())
}


In [41]:
val clientsJson = """


[
    {
        "Client_ID":"C001",
        "Need":"Grue",
        "Volume_t":13.73,
        "Volume_kg":13726.61,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8727114265,
        "Longitude":2.4152913367
    },
    {
        "Client_ID":"C002",
        "Need":"Enrobée",
        "Volume_t":16.51,
        "Volume_kg":16511.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8337749631,
        "Longitude":2.4624881962
    },
    {
        "Client_ID":"C003",
        "Need":"Grue",
        "Volume_t":14.97,
        "Volume_kg":14966.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8995203229,
        "Longitude":2.3830675345
    },
    {
        "Client_ID":"C004",
        "Need":"Grue",
        "Volume_t":16.41,
        "Volume_kg":16406.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8552772065,
        "Longitude":2.3719840732
    },
    {
        "Client_ID":"C005",
        "Need":"Waste",
        "Volume_t":10.42,
        "Volume_kg":10417.48,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8267392211,
        "Longitude":2.3840461925
    },
    {
        "Client_ID":"C006",
        "Need":"Enrobée",
        "Volume_t":17.52,
        "Volume_kg":17524.55,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8798084826,
        "Longitude":2.2785357718
    },
    {
        "Client_ID":"C007",
        "Need":"Enrobée",
        "Volume_t":17.49,
        "Volume_kg":17491.73,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8424366831,
        "Longitude":2.3312467777
    },
    {
        "Client_ID":"C008",
        "Need":"Enrobée",
        "Volume_t":13.61,
        "Volume_kg":13605.94,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8297649991,
        "Longitude":2.3169421334
    },
    {
        "Client_ID":"C009",
        "Need":"Enrobée",
        "Volume_t":14.16,
        "Volume_kg":14164.58,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8632249724,
        "Longitude":2.3568582611
    },
    {
        "Client_ID":"C010",
        "Need":"Waste",
        "Volume_t":12.89,
        "Volume_kg":12893.21,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8357622912,
        "Longitude":2.4148355918
    },
    {
        "Client_ID":"C011",
        "Need":"Waste",
        "Volume_t":13.35,
        "Volume_kg":13348.32,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.885563268,
        "Longitude":2.4053477927
    },
    {
        "Client_ID":"C012",
        "Need":"Waste",
        "Volume_t":10.18,
        "Volume_kg":10182.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8344111745,
        "Longitude":2.3409284004
    },
    {
        "Client_ID":"C013",
        "Need":"Waste",
        "Volume_t":18.69,
        "Volume_kg":18693.3,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8444315354,
        "Longitude":2.2819629252
    },
    {
        "Client_ID":"C014",
        "Need":"Waste",
        "Volume_t":14.21,
        "Volume_kg":14212.6,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8882465175,
        "Longitude":2.264484004
    },
    {
        "Client_ID":"C015",
        "Need":"Waste",
        "Volume_t":12.43,
        "Volume_kg":12430.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8567207885,
        "Longitude":2.2987460852
    },
    {
        "Client_ID":"C016",
        "Need":"Enrobée",
        "Volume_t":15.98,
        "Volume_kg":15981.13,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8360376647,
        "Longitude":2.3018284564
    },
    {
        "Client_ID":"C017",
        "Need":"Waste",
        "Volume_t":18.58,
        "Volume_kg":18577.18,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8563111256,
        "Longitude":2.4515215805
    },
    {
        "Client_ID":"C018",
        "Need":"Waste",
        "Volume_t":13.54,
        "Volume_kg":13536.02,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8843489829,
        "Longitude":2.3179995845
    },
    {
        "Client_ID":"C019",
        "Need":"Waste",
        "Volume_t":14.38,
        "Volume_kg":14380.32,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8277019945,
        "Longitude":2.3644008554
    },
    {
        "Client_ID":"C020",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.06,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.852508881,
        "Longitude":2.4209095852
    },
    {
        "Client_ID":"C021",
        "Need":"Waste",
        "Volume_t":15.71,
        "Volume_kg":15708.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8958473007,
        "Longitude":2.2283797347
    },
    {
        "Client_ID":"C022",
        "Need":"Waste",
        "Volume_t":17.75,
        "Volume_kg":17747.05,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8840654658,
        "Longitude":2.3356365874
    },
    {
        "Client_ID":"C023",
        "Need":"Waste",
        "Volume_t":11.92,
        "Volume_kg":11916.03,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8998471212,
        "Longitude":2.322535139
    },
    {
        "Client_ID":"C024",
        "Need":"Waste",
        "Volume_t":12.86,
        "Volume_kg":12862.85,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8930421522,
        "Longitude":2.3358789851
    },
    {
        "Client_ID":"C025",
        "Need":"Enrobée",
        "Volume_t":17.04,
        "Volume_kg":17044.63,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8509098728,
        "Longitude":2.2290630239
    },
    {
        "Client_ID":"C026",
        "Need":"Waste",
        "Volume_t":13.99,
        "Volume_kg":13990.61,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8945021227,
        "Longitude":2.2973676485
    },
    {
        "Client_ID":"C027",
        "Need":"Waste",
        "Volume_t":14.33,
        "Volume_kg":14333.43,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8570057014,
        "Longitude":2.2757384372
    },
    {
        "Client_ID":"C028",
        "Need":"Waste",
        "Volume_t":10.44,
        "Volume_kg":10438.7,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295147289,
        "Longitude":2.3976309301
    },
    {
        "Client_ID":"C029",
        "Need":"Enrobée",
        "Volume_t":15.05,
        "Volume_kg":15053.1,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8313453015,
        "Longitude":2.2891161603
    },
    {
        "Client_ID":"C030",
        "Need":"Waste",
        "Volume_t":17.0,
        "Volume_kg":17002.53,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8546398289,
        "Longitude":2.3367049267
    },
    {
        "Client_ID":"C031",
        "Need":"Waste",
        "Volume_t":19.75,
        "Volume_kg":19748.83,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8764464984,
        "Longitude":2.387878084
    },
    {
        "Client_ID":"C032",
        "Need":"Waste",
        "Volume_t":14.83,
        "Volume_kg":14832.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.888965554,
        "Longitude":2.2507665485
    },
    {
        "Client_ID":"C033",
        "Need":"Enrobée",
        "Volume_t":13.98,
        "Volume_kg":13976.26,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8634868191,
        "Longitude":2.463648015
    },
    {
        "Client_ID":"C034",
        "Need":"Waste",
        "Volume_t":15.17,
        "Volume_kg":15174.54,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8325553338,
        "Longitude":2.2648644966
    },
    {
        "Client_ID":"C035",
        "Need":"Waste",
        "Volume_t":11.15,
        "Volume_kg":11150.33,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8491311632,
        "Longitude":2.4134520825
    },
    {
        "Client_ID":"C036",
        "Need":"Enrobée",
        "Volume_t":16.84,
        "Volume_kg":16836.07,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8224189719,
        "Longitude":2.2367828582
    },
    {
        "Client_ID":"C037",
        "Need":"Waste",
        "Volume_t":14.0,
        "Volume_kg":13998.22,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.85201415,
        "Longitude":2.342531956
    },
    {
        "Client_ID":"C038",
        "Need":"Enrobée",
        "Volume_t":16.83,
        "Volume_kg":16825.14,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8511520307,
        "Longitude":2.4396337885
    },
    {
        "Client_ID":"C039",
        "Need":"Grue",
        "Volume_t":15.75,
        "Volume_kg":15745.92,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8288579155,
        "Longitude":2.3021950667
    },
    {
        "Client_ID":"C040",
        "Need":"Waste",
        "Volume_t":13.89,
        "Volume_kg":13892.92,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8372274366,
        "Longitude":2.4186360279
    },
    {
        "Client_ID":"C041",
        "Need":"Grue",
        "Volume_t":10.22,
        "Volume_kg":10215.16,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8986725908,
        "Longitude":2.2921971364
    },
    {
        "Client_ID":"C042",
        "Need":"Grue",
        "Volume_t":10.03,
        "Volume_kg":10031.65,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8598342054,
        "Longitude":2.3115851055
    },
    {
        "Client_ID":"C043",
        "Need":"Waste",
        "Volume_t":19.93,
        "Volume_kg":19929.27,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8210212275,
        "Longitude":2.2962086898
    },
    {
        "Client_ID":"C044",
        "Need":"Waste",
        "Volume_t":11.96,
        "Volume_kg":11960.49,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Afternoon",
        "Time_Window_Start":"13:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8762368303,
        "Longitude":2.2660315935
    },
    {
        "Client_ID":"C045",
        "Need":"Enrobée",
        "Volume_t":15.01,
        "Volume_kg":15014.12,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8872887819,
        "Longitude":2.4086223577
    },
    {
        "Client_ID":"C046",
        "Need":"Enrobée",
        "Volume_t":13.72,
        "Volume_kg":13724.74,
        "Access_Difficulty":"No",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8674790585,
        "Longitude":2.313550389
    },
    {
        "Client_ID":"C047",
        "Need":"Waste",
        "Volume_t":19.88,
        "Volume_kg":19877.12,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"Morning",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"12:00",
        "Latitude":48.8753450245,
        "Longitude":2.2514940748
    },
    {
        "Client_ID":"C048",
        "Need":"Waste",
        "Volume_t":17.63,
        "Volume_kg":17626.71,
        "Access_Difficulty":"Yes",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8295176506,
        "Longitude":2.2938424521
    },
    {
        "Client_ID":"C049",
        "Need":"Enrobée",
        "Volume_t":17.72,
        "Volume_kg":17715.78,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8431161596,
        "Longitude":2.3191251583
    },
    {
        "Client_ID":"C050",
        "Need":"Waste",
        "Volume_t":18.67,
        "Volume_kg":18668.69,
        "Access_Difficulty":"No",
        "Time_Window_Label":"All Day",
        "Time_Window_Start":"08:00",
        "Time_Window_End":"17:00",
        "Latitude":48.8664786498,
        "Longitude":2.2519575393
    }
]

""".trimIndent()


In [42]:
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()

val clients: List<Client> = mapper.readValue(clientsJson)

println("Loaded ${clients.size} clients:")
clients.forEach { println(it) }


Loaded 50 clients:
C001
C002
C003
C004
C005
C006
C007
C008
C009
C010
C011
C012
C013
C014
C015
C016
C017
C018
C019
C020
C021
C022
C023
C024
C025
C026
C027
C028
C029
C030
C031
C032
C033
C034
C035
C036
C037
C038
C039
C040
C041
C042
C043
C044
C045
C046
C047
C048
C049
C050


In [36]:
// Définition des équipements disponibles
val equipments = listOf(
    Equipment("Par bennes 10 m3", capacityKg = 10000.0),
    Equipment("Par bennes 20 m3", capacityKg = 20000.0),
    Equipment("Par pont 13 m3", capacityKg = 13000.0),
    Equipment("Par thermos 15 à 18 to", capacityKg = 18000.0),
    Equipment("Crane", capacityKg = 8500.0),
    Equipment("Remorque tandem", capacityKg = 8500.0)
)


In [37]:
// Définition des camions disponibles
val trucks = listOf(
    Truck(
        name = "Truck 1",
        type = "Multilift 4 essieux",
        homeLocation = Location(48.8566, 2.3522), // Paris
        capacityKg = 32000.0,
        canAccessDifficultAreas = false,
        canEquipRemorque = false,
        possibleEquipments = equipments.filter { it.name != "Crane" && it.name != "Remorque tandem" }
    ),
    Truck(
        name = "Truck 2",
        type = "Multilift 4 essieux Tridem",
        homeLocation = Location(48.8566, 2.3522),
        capacityKg = 32000.0,
        canAccessDifficultAreas = true,
        canEquipRemorque = false,
        possibleEquipments = equipments.filter { it.name != "Crane" && it.name != "Remorque tandem" }
    ),
    Truck(
        name = "Truck 3",
        type = "Camion-grue 4 essieux",
        homeLocation = Location(48.8566, 2.3522),
        capacityKg = 8500.0,
        canAccessDifficultAreas = false,
        canEquipRemorque = true,
        possibleEquipments = equipments.filter { it.name == "Crane" || it.name == "Remorque tandem" }
    )
)


In [38]:
// Importations nécessaires pour les annotations de planification
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId

@PlanningEntity
data class Client(
    @PlanningId
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" ou "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Conversion de "Yes"/"No" en booléen
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    // Variable de planification : le camion assigné au client
    @PlanningVariable(valueRangeProviderRefs = ["truckRange"])
    var truck: Truck? = null

    override fun toString(): String = clientId
}
